In [2]:
! pip install torch transformers onnx onnxruntime onnxruntime-tools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 84.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 86.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.7 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
import os
import torch
from transformers import AutoTokenizer, AutoModel
from onnxruntime.quantization import quantize_dynamic, QuantType

MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
ONNX_DIR = "/content/drive/MyDrive/Colab Notebooks/model_export/onnx"
FP32_MODEL = os.path.join(ONNX_DIR, "model.onnx")
INT8_MODEL = os.path.join(ONNX_DIR, "model-int8.onnx")



In [6]:
!pip install -U onnx onnxscript

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 693.4/693.4 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.1/133.1 kB 7.6 MB/s eta 0:00:00


In [7]:
os.makedirs(ONNX_DIR, exist_ok=True)

print("Loading pretrained MiniLM...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)
model.eval()

# Dummy input for export
dummy_text = "This is a test sentence for ONNX export."
inputs = tokenizer(dummy_text, return_tensors="pt")

print("Exporting FP32 ONNX model...")
torch.onnx.export(
    model,
    (inputs["input_ids"], inputs["attention_mask"]),
    FP32_MODEL,
    input_names=["input_ids", "attention_mask"],
    output_names=["last_hidden_state"],
    dynamic_axes={
        "input_ids": {0: "batch", 1: "sequence"},
        "attention_mask": {0: "batch", 1: "sequence"},
        "last_hidden_state": {0: "batch", 1: "sequence"},
    },
    opset_version=17,
)

print("FP32 ONNX saved:", FP32_MODEL)

print("Quantizing to INT8...")
quantize_dynamic(
    model_input=FP32_MODEL,
    model_output=INT8_MODEL,
    weight_type=QuantType.QInt8,
)

print("INT8 ONNX saved:", INT8_MODEL)

print("Done ✅")


Loading pretrained MiniLM...
Exporting FP32 ONNX model...


/tmp/ipython-input-1671631514.py:13: UserWarning: # 'dynamic_axes' is not recommended when dynamo=True, and may lead to 'torch._dynamo.exc.UserError: Constraints violated.' Supply the 'dynamic_shapes' argument instead if export is unsuccessful.
  torch.onnx.export(
W1227 08:27:46.693000 693 torch/onnx/_internal/exporter/_compat.py:114] Setting ONNX exporter to use operator set version 18 because the requested opset_version 17 is a lower version than we have implementations for. Automatic version conversion will be performed, which may not be successful at converting to the requested version. If version conversion is unsuccessful, the opset version of the exported model will be kept at 18. Please consider setting opset_version >=18 to leverage latest ONNX features


[torch.onnx] Obtain model graph for `BertModel([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `BertModel([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...


[torch.onnx] Translate the graph into ONNX... ✅
Applied 42 of general pattern rewrite rules.
FP32 ONNX saved: /content/drive/MyDrive/Colab Notebooks/model_export/onnx/model.onnx
Quantizing to INT8...


INT8 ONNX saved: /content/drive/MyDrive/Colab Notebooks/model_export/onnx/model-int8.onnx
Done ✅
